In [ ]:
# Load required packages
library(sf)
library(mkde)
library(terra)
library(ggplot2)
library(rgl)
library(gganimate)
library(scales)

In [ ]:
# This software is Copyright 2022 The Regents of the University of California.
# All Rights Reserved. Permission to copy, modify, and distribute this software
# and its documentation for educational, research and non-profit purposes,
# without fee, and without a written agreement is hereby granted, provided that
# this entire copyright appear in all copies. Permission to make commercial use
# of this software may be obtained by contacting:
# 
# Office of Innovation and Commercialization
# 9500 Gilman Drive, Mail Code 0910
# University of California
# La Jolla, CA 92093-0910
# (858) 534-5815
# invent@ucsd.edu
#
# This software program and documentation are copyrighted by The Regents of the
# University of California. The software program and documentation are supplied
# “as is”, without any accompanying services from The Regents. The Regents does
# not warrant that the operation of the program will be uninterrupted or
# error-free. The end-user understands that the program was developed for
# research purposes and is advised not to rely exclusively on the program for
# any reason.
# 
# IN NO EVENT SHALL THE UNIVERSITY OF CALIFORNIA BE LIABLE TO ANY PARTY FOR
# DIRECT, INDIRECT, SPECIAL, INCIDENTAL, OR CONSEQUENTIAL DAMAGES, INCLUDING
# LOST PROFITS, ARISING OUT OF THE USE OF THIS SOFTWARE AND ITS DOCUMENTATION,
# EVEN IF THE UNIVERSITY OF CALIFORNIA HAS BEEN ADVISED OF THE POSSIBILITY OF
# SUCH DAMAGE. THE UNIVERSITY OF CALIFORNIA SPECIFICALLY DISCLAIMS ANY
# WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF
# MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. THE SOFTWARE PROVIDED
# HEREUNDER IS ON AN “AS IS” BASIS, AND THE UNIVERSITY OF CALIFORNIA HAS NO
# OBLIGATIONS TO PROVIDE MAINTENANCE, SUPPORT, UPDATES, ENHANCEMENTS, OR
# MODIFICATIONS.

# Calculate rasters for each indivdual in a dataframe using mkde
calculateRaster2D <- function (gpsdata, id, sig2obs, t.max, cell.sz, buffer) {

  x <- gpsdata[which(gpsdata$id == id), "x"]
  y <- gpsdata[which(gpsdata$id == id), "y"]
  t <- gpsdata[which(gpsdata$id == id), "time"]

  xmin = min(x) - buffer
  xmax = max(x) + buffer 
  ymin = min(y) - buffer
  ymax = max(y) + buffer

  xrange <- xmax - xmin
  yrange <- ymax - ymin

  nx <- as.integer(xrange/cell.sz)
  ny <- as.integer(yrange/cell.sz)
  
  mv.dat <- initializeMovementData(t, x, y, sig2obs = sig2obs, t.max = t.max)
  mkde.obj <- initializeMKDE2D(xmin, cell.sz, nx, ymin, cell.sz, ny)
  dens.res <- initializeDensity(mkde.obj, mv.dat)
  mkde.obj <- dens.res$mkde.obj
  
  return (mkde.obj)
  
}

# mkdeToTerra converts an mkde object to a SpatRaster
mkdeToTerra <- function(mkde.obj) {
  
  sx <- mkde.obj$x[2] - mkde.obj$x[1]
  sy <- mkde.obj$y[2] - mkde.obj$y[1]
  
  if (mkde.obj$dimension == 2 | mkde.obj$dimension == 2.5) {
    
    # Calculate the extent using the calculated values
    extent <- c(min(mkde.obj$x) - 0.5 * sx, max(mkde.obj$x) + 0.5 * sx,
                min(mkde.obj$y) - 0.5 * sy, max(mkde.obj$y) + 0.5 * sy)
    
    rst <- terra::rast(nrow = mkde.obj$ny, ncol = mkde.obj$nx, xmin = extent[1], xmax = extent[2],
                ymin = extent[3], ymax = extent[4], crs = st_crs("+proj=longlat"))
    
    # Transpose and then flip vertically by reversing rows
    d_matrix <- base::t(matrix(mkde.obj$d[,,1], nrow = mkde.obj$nx, ncol = mkde.obj$ny))
    d_flipped <- d_matrix[base::nrow(d_matrix):1, ]
    
    # Assign the flipped matrix to the raster
    terra::values(rst) <- d_flipped
    
    return(rst)
    
  } 
  
  else {
    
    # Not able to test this
    stop("Only 2D and 2.5D MKDE objects are supported")
    
  }
  
}


In [ ]:
# Function to read data from a text file into a dataframe
read_data <- function(file_path) {
  df <- read.table(file_path, header = TRUE)  # Assuming the file has column headings
  return(df)
}


# Assign a standard ID number like 0 to all data points
assign_id <- function(df) {
  df$id <- 1
  return(df)
}

# Call the provided functions to process the data
process_data <- function(file_path) {
  # Read data from file
  df <- read_data(file_path)
  
  # Assign a standard ID number
  df <- assign_id(df)
  
  return(df)
}

In [ ]:
# Example usage
file_path <-Sys.getenv("DATASET_TXT_1")

# Update this with the path to your data file
processed_data <- process_data(file_path)
#print(processed_data)

mkdeobj <- calculateRaster2D(processed_data, 1, 25, 185, 60, 1000)

In [ ]:
spatraster <- mkdeToTerra(mkdeobj)
print(spatraster)
plot(spatraster)

In [ ]:
# Convert spatraster object to dataframe
raster_df <- as.data.frame(spatraster, xy = TRUE)

# Define color palette (adjust as needed)
my_palette <- viridisLite::viridis(100)

# Manually adjust the scaling range
scale_limits <- c(0, max(raster_df$lyr.1, na.rm = TRUE) * 0.3)  # Adjust the multiplier (0.8) as needed

# Plot using ggplot2 with adjusted scaling and no legend
ggplot(raster_df, aes(x = x, y = y, fill = lyr.1)) +
  geom_raster(interpolate = TRUE) +
  scale_fill_gradientn(colors = my_palette, limits = scale_limits, na.value = "transparent") +
  labs(x = "X Axis Label", y = "Y Axis Label", title = "Raster Visualization") +
  theme_minimal()  # Adjust theme as needed
